## Часть 1
Напишите скрипт, который скачивает все данные прошедших губернаторских выборов для всех избирательных участков. 

Входная точка [по ссылке](http://www.st-petersburg.vybory.izbirkom.ru/region/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222). Затем нужно перейти на сайты территориальных избирательных комиссий. Результаты нужно сохранить в  `cvs`-файл, `sqlite` базе данных или `parquet`-файле. В итоге должна получиться таблица с полями:
- название ТИК (территориальная избирательная комиссия, некоторое объединение избирательных участков на определенной территории)
- номер УИК (избирательный участок)
- 14 стандартных полей из итогового протокола

Скрипт должен быть в отдельном `*.py` файле или в отдельном `jupyter`-ноутбуке. 

In [159]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import sqlite3
import json

In [160]:
path = 'http://www.st-petersburg.vybory.izbirkom.ru/region/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222'
response = requests.get(path)

In [161]:
soup = BeautifulSoup(response.text)
connection = sqlite3.connect("Statistic.db")

In [162]:
table = soup.find('table',{ 'style' : 'width:100%;border-color:#000000' })
links_to_TEC=table.findAll('a', attrs={'href': re.compile("^http://")})

In [163]:
def preprocess_data(result_table):
    
    def preprocess_named_statistic(str):
        list=str.split(" ")
        list[0] = int(list[0])
        list[1] = float(list[1].strip('%'))
        return json.dumps(list)

    for i in range(2,13):
        result_table[list(result_table)[i]]=result_table[list(result_table)[i]].astype(int)

    for i in range (13,16):
        result_table[list(result_table)[i]] =  result_table[list(result_table)[i]].map(lambda x: preprocess_named_statistic(x))
    
    return result_table

In [164]:
for TEC in links_to_TEC:
    response=requests.get(TEC.get('href'))
    soupL=BeautifulSoup(response.text)
    
    table = soupL.find('table',{ 'style' : 'width:100%;border-color:#000000' })
        
    if table != None:
        # считываем таблицу из html страницы:
        df = pd.read_html(str(table.findAll('td')),header=0)
        # парсим нужным нам образом - склеиваем в одну и переворачивам для удобства хранения:
        result_table=df[0].dropna().join(df[1].dropna()).T
        # переименовываем колонки для наглядности данных и возвращаем индексы:
        result_table.rename(index={'Unnamed: 1' : 'Статистика по',},inplace = True)
        result_table.reset_index(inplace=True)
        result_table.columns = result_table.iloc[1]
        result_table=result_table.drop(result_table.index[1]).drop(result_table.index[0])
        # добавляем ТИК к данным  обновляем индексы после дропа лишнего:
        new_col=[TEC.text for i in range(result_table.shape[0])]
        result_table.insert(loc=0, column='ТИК', value=new_col)
        result_table.reset_index(drop=True,inplace=True)
        # сохраняем в БДшку:
        preprocess_data(result_table).to_sql("t_statistic", connection, if_exists="append")